# Заполним пропуски

Пропуски есть в near_metro и class

# Near metro
## Заполним через api yandex "поиск по организациям"

In [1]:
import json
import geopy.distance as distance
import pandas as pd
import requests

In [2]:
data = pd.read_csv('../result.csv')

In [3]:
def get_near_metro(lon, lat, api_key):
    req_text = 'https://search-maps.yandex.ru/v1/?text=метро&ll=' + str(lon) + "," + str(lat) + '&lang=ru_RU&results=1&apikey=' + api_key
    ans = json.loads(requests.get(req_text).text)
    try:
        metro_lon, metro_lat = ans['features'][0]['geometry']['coordinates']
        result = distance.vincenty((lon, lat), (metro_lon, metro_lat)).m
    except:
        result = np.nan
    return result

In [4]:
null_metro = data[data['near_metro'].isnull()]

In [5]:
null_metro.head(2)

,Unnamed: 0,size,class,address,price,pledge,descr,metros,near_metro,url,parking,internet,meal,security,comfort,lat,lon,distance
39,1002926616,57.0,NaN,"Петровско-Разумовская, Лихоборская наб 7",685,0,Сдаются офис склад на территории бывшего про...,NaN,NaN,https://www.avito.ru/moskva/kommercheskaya_ned...,1,0,0,0,0,55.854411,37.529070,0.20401
56,1690633446,39.0,NaN,"Автозаводская, Кожуховский 2-й проезд 29к2с...",1085,0,ДО КОНЦА 2018 ГОДА МЕСЯЦ АРЕНДЫ ЭТОГО ПОМЕЩЕ...,NaN,NaN,https://www.avito.ru/moskva/kommercheskaya_ned...,1,0,0,1,1,55.698391,37.658248,0.09784


In [6]:
with open('../apies/yandex_organization_search_api.txt') as f:
    apikey = f.readline()

Через API можно отправлять только 500 запросов, раз в сутки

In [7]:
for index, row in null_metro.iterrows():
    null_metro.at[index, 'near_metro'] = get_near_metro(row['lon'], row['lat'], apikey)

In [10]:
data[data['near_metro'].isnull()] = null_metro
data.to_csv('got_near_metro.csv')

# Class

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('got_near_metro.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3155 entries, 0 to 3154
Data columns (total 14 columns):
Unnamed: 0    3155 non-null int64
size          3155 non-null float64
class         1954 non-null object
price         3155 non-null int64
pledge        3155 non-null int64
near_metro    3155 non-null float64
parking       3155 non-null int64
internet      3155 non-null int64
meal          3155 non-null int64
security      3155 non-null int64
comfort       3155 non-null int64
lat           3155 non-null float64
lon           3155 non-null float64
distance      3155 non-null float64
dtypes: float64(5), int64(8), object(1)
memory usage: 345.2+ KB


In [5]:
data.head(5)

,Unnamed: 0,size,class,price,pledge,near_metro,parking,internet,meal,security,comfort,lat,lon,distance
0,0,28.8,NaN,1354,39000,700.0,0,0,0,0,1,55.745093,37.664815,0.09581
1,1,588.2,NaN,850,500000,700.0,1,0,0,0,1,55.745093,37.664815,0.09581
2,2,45.0,NaN,1111,50000,200.0,1,0,0,1,0,55.759659,37.752850,0.27184
3,3,100.0,NaN,1700,170000,700.0,1,0,0,1,0,55.569026,37.588647,0.19060
4,4,190.0,NaN,789,150000,1200.0,0,1,0,0,0,55.757496,37.802086,0.37023


In [8]:
#data = data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'address', 'descr', 'metros', 'url'], axis=1)
# ты мне, видимо, другую версию кинул с меньшим числом столбцов
data = data.drop(['Unnamed: 0'], axis=1)
data.head(5)

,size,class,price,pledge,near_metro,parking,internet,meal,security,comfort,lat,lon,distance
0,28.8,NaN,1354,39000,700.0,0,0,0,0,1,55.745093,37.664815,0.09581
1,588.2,NaN,850,500000,700.0,1,0,0,0,1,55.745093,37.664815,0.09581
2,45.0,NaN,1111,50000,200.0,1,0,0,1,0,55.759659,37.752850,0.27184
3,100.0,NaN,1700,170000,700.0,1,0,0,1,0,55.569026,37.588647,0.19060
4,190.0,NaN,789,150000,1200.0,0,1,0,0,0,55.757496,37.802086,0.37023


Цена является целевой переменной, поэтому использовать ее для заполнения пропусков нельзя.

Нужно разделить на обучающую и целевую выборку.

In [9]:
null_classes = data[data['class'].isnull()].drop(['price'], axis=1)
not_null_classes = data[~data['class'].isnull()].drop(['price'], axis=1)

Заполним Nan-ы knn - ом.

In [10]:
null_classes.head(5)

,size,class,pledge,near_metro,parking,internet,meal,security,comfort,lat,lon,distance
0,28.8,NaN,39000,700.0,0,0,0,0,1,55.745093,37.664815,0.09581
1,588.2,NaN,500000,700.0,1,0,0,0,1,55.745093,37.664815,0.09581
2,45.0,NaN,50000,200.0,1,0,0,1,0,55.759659,37.752850,0.27184
3,100.0,NaN,170000,700.0,1,0,0,1,0,55.569026,37.588647,0.19060
4,190.0,NaN,150000,1200.0,0,1,0,0,0,55.757496,37.802086,0.37023


In [12]:
not_null_classes.head(5)

,size,class,pledge,near_metro,parking,internet,meal,security,comfort,lat,lon,distance
5,31.5,b,33000,300.0,1,0,0,0,1,55.635396,37.436661,0.37875
6,143.5,b,191333,500.0,1,0,1,1,7,55.776198,37.675604,0.11989
8,20.4,b,47000,800.0,1,1,1,1,7,55.722918,37.468793,0.29774
11,1021.0,a,0,300.0,0,0,0,1,6,55.761983,37.606846,0.02309
13,72.3,b,0,500.0,1,0,1,1,4,55.720489,37.615452,0.03067


In [48]:
ohe_features = ['parking', 'internet', 'meal', 'security']
X_data = not_null_classes.drop(['class'], axis=1)
X_data.drop(ohe_features, axis=1, inplace=True)
y_data = not_null_classes['class'].copy()
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)
for feature in ohe_features:
    X_data = np.append(X_data, np.array([not_null_classes[feature]]).T, axis=1)
print(pd.DataFrame(X_data).head(5))

         0         1         2         3         4         5         6    7   \
0 -0.294019 -0.230755 -0.850470 -0.802013 -1.708711 -1.948412  2.011791  1.0   
1 -0.077594  0.077495 -0.510802  1.752402  0.421493  0.679632 -0.371952  1.0   
2 -0.315468 -0.203499 -0.001300  1.752402 -0.384584 -1.595005  1.265801  1.0   
3  1.618061 -0.295001 -0.850470  1.326666  0.206433 -0.076612 -1.263346  0.0   
4 -0.215178 -0.295001 -0.510802  0.475194 -0.421333  0.018042 -1.193545  1.0   

    8    9    10  
0  0.0  0.0  0.0  
1  0.0  1.0  1.0  
2  1.0  1.0  1.0  
3  0.0  0.0  1.0  
4  0.0  1.0  1.0  


In [14]:
# X_data = not_null_classes.drop(['class'], axis=1)
# y_data = not_null_classes['class'].copy()
# scaler = StandardScaler()
# X_data = scaler.fit_transform(X_data)
# X_data

        size  pledge  near_metro  parking  internet  meal  security  comfort  \
5       31.5   33000       300.0        1         0     0         0        1   
6      143.5  191333       500.0        1         0     1         1        7   
8       20.4   47000       800.0        1         1     1         1        7   
11    1021.0       0       300.0        0         0     0         1        6   
13      72.3       0       500.0        1         0     1         1        4   
14     114.0       0       100.0        1         1     0         1        3   
15     163.6  572600       100.0        1         1     0         0        2   
17      25.0       0      1500.0        1         0     1         1        2   
19     155.0       0       100.0        1         0     0         1        4   
21     144.4  556000       100.0        0         0     1         0        0   
22      71.0   97000       700.0        0         0     0         0        2   
29    1387.0    2500       800.0        

array([[-2.94019133e-01, -2.30754993e-01, -8.50470069e-01, ...,
        -1.70871113e+00, -1.94841224e+00,  2.01179132e+00],
       [-7.75937377e-02,  7.74952482e-02, -5.10802141e-01, ...,
         4.21492520e-01,  6.79631533e-01, -3.71951819e-01],
       [-3.15468436e-01, -2.03499125e-01, -1.30024871e-03, ...,
        -3.84584458e-01, -1.59500452e+00,  1.26580109e+00],
       ...,
       [-3.01555375e-01, -2.56064014e-01, -8.50470069e-01, ...,
        -6.04924269e-01,  4.27774084e-01, -7.10092312e-01],
       [ 7.02325369e-02, -2.95000968e-01,  5.08201644e-01, ...,
        -2.01991615e+00, -2.90314959e-01, -1.08769485e-01],
       [ 1.53110396e+00, -2.44382927e-01, -3.40968177e-01, ...,
         5.83403852e-01,  6.38529682e-01, -4.12746002e-01]])

Можно написать свой скелер, чтобы он не скейлил категориальные признаки.

In [41]:
clf = KNeighborsClassifier()
scores = cross_val_score(clf, X_data, y_data, cv=15)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.72932331 0.72180451 0.73484848 0.77272727 0.72519084 0.66412214
 0.73076923 0.74418605 0.65116279 0.66666667 0.75193798 0.75968992
 0.75968992 0.70542636 0.80620155]
Accuracy: 0.73 (+/- 0.08)


In [42]:
clf.fit(X_data, y_data)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [49]:
Target_data = null_classes.drop(['class'], axis=1)
Target_data.drop(ohe_features, axis=1, inplace=True)
Target_data = pd.DataFrame(scaler.transform(Target_data))
for feature in ohe_features:
    Target_data = np.append(Target_data, np.array([null_classes[feature]]).T, axis=1)

In [50]:
print(pd.DataFrame(X_data).head(5))

         0         1         2         3         4         5         6    7   \
0 -0.294019 -0.230755 -0.850470 -0.802013 -1.708711 -1.948412  2.011791  1.0   
1 -0.077594  0.077495 -0.510802  1.752402  0.421493  0.679632 -0.371952  1.0   
2 -0.315468 -0.203499 -0.001300  1.752402 -0.384584 -1.595005  1.265801  1.0   
3  1.618061 -0.295001 -0.850470  1.326666  0.206433 -0.076612 -1.263346  0.0   
4 -0.215178 -0.295001 -0.510802  0.475194 -0.421333  0.018042 -1.193545  1.0   

    8    9    10  
0  0.0  0.0  0.0  
1  0.0  1.0  1.0  
2  1.0  1.0  1.0  
3  0.0  0.0  1.0  
4  0.0  1.0  1.0  


In [51]:
#Target_data['class'] = clf.predict(Target_data)
classes = clf.predict(Target_data)

In [52]:
classes

array(['b', 'b', 'b', ..., 'b', 'b', 'b'], dtype=object)

In [53]:
data.loc[data['class'].isnull(), 'class'] = classes

In [54]:
data.head(5)

,size,class,price,pledge,near_metro,parking,internet,meal,security,comfort,lat,lon,distance
0,28.8,b,1354,39000,700.0,0,0,0,0,1,55.745093,37.664815,0.09581
1,588.2,b,850,500000,700.0,1,0,0,0,1,55.745093,37.664815,0.09581
2,45.0,b,1111,50000,200.0,1,0,0,1,0,55.759659,37.752850,0.27184
3,100.0,b,1700,170000,700.0,1,0,0,1,0,55.569026,37.588647,0.19060
4,190.0,c,789,150000,1200.0,0,1,0,0,0,55.757496,37.802086,0.37023


In [55]:
data.to_csv('data_with_classes_and_metros.csv')